hemispheric only is without filtering (at first) to just see the hemispheric plots

-> my theory is that filtering is messing with the cellID configuration, causing it to not plot correctly!


In [1]:
#Chunk that sloads necessary packages & sets working environment to where the jupyterlab notebook file is 
%load_ext autoreload
%autoreload 2
import gnssvod as gv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pdb
import importlib
import zipfile
import os
import xarray as xr
import glob
import shutil
from gnssvod.hemistats.hemistats import hemibuild
#import georinex as gr
from matplotlib.collections import PatchCollection
import matplotlib.dates as mdates
import shutil
from matplotlib import rcParams
from datetime import datetime, timedelta

In [2]:
#pairing and concatenating--- starting by establishing path to .nc files per station
##here MACROCOSM 5 & 1 are references
# MACROCOSM 3 is located in- gutter
#MACROCOSM 2 is located in- under exclosure
#MACROCOSM-7 is located in- coffe1
pattern={'MACROCOSM-1':'/Volumes/coraldelmar/all_data/MACROCOSM-1/nc/*.nc',
        'MACROCOSM-2':'/Volumes/coraldelmar/all_data/MACROCOSM-2/nc/*.nc',
        'MACROCOSM-3':'/Volumes/coraldelmar/all_data/MACROCOSM-3/nc/*.nc',
        'MACROCOSM-5':'/Volumes/coraldelmar/all_data/MACROCOSM-5/nc/*.nc',
        'MACROCOSM-7':'/Volumes/coraldelmar/all_data/MACROCOSM-7/nc/*.nc'}

In [3]:
startday = start=pd.to_datetime('01-01-2025',format='%d-%m-%Y')
timeintervals=pd.interval_range(start=startday, periods=150, freq='D', closed='left')
timeintervals

IntervalIndex([[2025-01-01 00:00:00, 2025-01-02 00:00:00),
               [2025-01-02 00:00:00, 2025-01-03 00:00:00),
               [2025-01-03 00:00:00, 2025-01-04 00:00:00),
               [2025-01-04 00:00:00, 2025-01-05 00:00:00),
               [2025-01-05 00:00:00, 2025-01-06 00:00:00),
               [2025-01-06 00:00:00, 2025-01-07 00:00:00),
               [2025-01-07 00:00:00, 2025-01-08 00:00:00),
               [2025-01-08 00:00:00, 2025-01-09 00:00:00),
               [2025-01-09 00:00:00, 2025-01-10 00:00:00),
               [2025-01-10 00:00:00, 2025-01-11 00:00:00),
               ...
               [2025-05-21 00:00:00, 2025-05-22 00:00:00),
               [2025-05-22 00:00:00, 2025-05-23 00:00:00),
               [2025-05-23 00:00:00, 2025-05-24 00:00:00),
               [2025-05-24 00:00:00, 2025-05-25 00:00:00),
               [2025-05-25 00:00:00, 2025-05-26 00:00:00),
               [2025-05-26 00:00:00, 2025-05-27 00:00:00),
               [2025-05-27 00:00:00, 

In [4]:
ds=xr.open_mfdataset('/Volumes/coraldelmar/all_data/MACROCOSM-5.7/*.nc',combine='nested',concat_dim='Epoch')
ds

<xarray.Dataset> Size: 7GB
Dimensions:    (Station: 2, Epoch: 580480, SV: 120)
Coordinates:
  * Station    (Station) <U11 88B 'MACROCOSM-5' 'MACROCOSM-7'
  * SV         (SV) <U3 1kB 'C11' 'C12' 'C14' 'C19' ... 'R24' 'R25' 'R26' 'R27'
  * Epoch      (Epoch) datetime64[ns] 5MB 2023-05-08T12:18:30 ... 2025-07-21T...
Data variables:
    S1C        (Station, Epoch, SV) float64 1GB dask.array<chunksize=(1, 11, 3), meta=np.ndarray>
    S1X        (Station, Epoch, SV) float64 1GB dask.array<chunksize=(1, 11, 3), meta=np.ndarray>
    S2C        (Station, Epoch, SV) float64 1GB dask.array<chunksize=(1, 11, 3), meta=np.ndarray>
    S2X        (Station, Epoch, SV) float64 1GB dask.array<chunksize=(1, 11, 3), meta=np.ndarray>
    Azimuth    (Station, Epoch, SV) float64 1GB dask.array<chunksize=(1, 11, 3), meta=np.ndarray>
    Elevation  (Station, Epoch, SV) float64 1GB dask.array<chunksize=(1, 11, 3), meta=np.ndarray>

In [ ]:
df=ds.to_dataframe().dropna(how='all').reorder_levels(["Station","Epoch","SV"]).sort_index()
df

In [ ]:
hemi=gv.hemibuild(4)
hemi

In [ ]:
hemi.grid

In [ ]:
patches=hemi.patches()
patches

In [ ]:
newdf=hemi.add_CellID(df)

In [ ]:
newdf

In [ ]:
hemi_average=newdf.groupby(['CellID','Station']).mean()
hemi_average

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,10),subplot_kw=dict(projection='polar'))

station_names = df.index.get_level_values('Station').unique()
for i, iname in enumerate(station_names):
    # associate the mean values to the patches, join inner will drop patches with no data, making plotting slightly faster
    ipatches = pd.concat([patches,hemi_average.xs(iname, level='Station')],join='inner',axis=1)
    # plotting with colored patches
    pc = PatchCollection(ipatches.Patches,array=ipatches.S1C,edgecolor='face', linewidth=1)
    pc.set_clim([25,50])
    ax[i].add_collection(pc)
    ax[i].set_rlim([0,90])
    ax[i].set_theta_zero_location("N")
    ax[i].set_title(iname)

plt.colorbar(pc, ax=ax, location='bottom', shrink=.5, pad=0.05, label='SNR (L1)')
#plt.savefig('figures/illustration_snr.png',facecolor='white', transparent=False,bbox_inches='tight')

In [ ]:
pattern='/Volumes/coraldelmar/all_data/MACROCOSM-5.7/*.nc'
pairings={'MACROCOSM-5.7':('MACROCOSM-5','MACROCOSM-7')}
bands={'VOD1':['S1C','S1X'],'VOD2':['S2C','S2X']}
vod=gv.calc_vod(pattern,pairings,bands)
vod=vod['MACROCOSM-5.7']
vod

In [ ]:
hemi=gv.hemibuild(2)
patches=hemi.patches()
vod=hemi.add_CellID(vod).drop(columns=['Azimuth','Elevation'])
vod_avg=vod.groupby(['CellID']).agg(['mean','std','count'])
vod_avg.columns=["_".join(x) for x in vod_avg.columns.to_flat_index()]
vod_avg

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(17,14),subplot_kw=dict(projection='polar'))

vod_names = ['VOD1','VOD2']
for i, iname in enumerate(vod_names):
    # associate the mean values to the patches, join inner will drop patches with no data, making plotting slightly faster
    ipatches = pd.concat([patches,vod_avg[f"{iname}_mean"]],join='inner',axis=1)
    # plotting with colored patches
    pc = PatchCollection(ipatches.Patches,array=ipatches[f"{iname}_mean"],edgecolor='face', linewidth=1)
    pc.set_clim([-0.1,3])
    ax[i].add_collection(pc)
    ax[i].set_rlim([0,90])
    ax[i].set_theta_zero_location("N")
    ax[i].set_theta_direction(-1)
    ax[i].set_title(iname)

plt.colorbar(pc, ax=ax, location='bottom', shrink=.5, pad=0.05, label='GNSS-VOD')

In [ ]:
# Set global font to Times New Roman and label size to 14
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["axes.labelsize"] = 14
plt.rcParams["xtick.labelsize"] = 14
plt.rcParams["ytick.labelsize"] = 14
plt.rcParams["legend.fontsize"] = 14
plt.rcParams["figure.titlesize"] = 14
plt.rcParams["axes.titlesize"] = 14
fig, ax = plt.subplots(figsize=(8, 7), subplot_kw=dict(projection='polar'))

#
fig, ax = plt.subplots(figsize=(8, 7), subplot_kw=dict(projection='polar'))

iname = 'VOD1'

# Associate the mean values to the patches
ipatches = pd.concat([patches, vod_avg[f"{iname}_mean"]], join='inner', axis=1)

# Plot with colored patches
pc = PatchCollection(ipatches.Patches, array=ipatches[f"{iname}_mean"], edgecolor='face', linewidth=1)
pc.set_clim([-0.1, 3])
ax.add_collection(pc)

# Configure the polar plot
ax.set_rlim([0, 90])
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
ax.set_title("Site 1a", y=1.08)

# Add colorbar
plt.colorbar(pc, ax=ax, location='bottom', shrink=.6, pad=0.1, label='GNSS-VOD')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import pandas as pd

# Set global font to Times New Roman and label size to 14
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["axes.labelsize"] = 14
plt.rcParams["xtick.labelsize"] = 14
plt.rcParams["ytick.labelsize"] = 14
plt.rcParams["legend.fontsize"] = 14
plt.rcParams["figure.titlesize"] = 14
plt.rcParams["axes.titlesize"] = 14

# Create figure and polar subplot
fig, ax = plt.subplots(figsize=(8, 7), subplot_kw=dict(projection='polar'))

# Select variable name
iname = 'VOD1'

# Combine patches and VOD data
ipatches = pd.concat([patches, vod_avg[f"{iname}_mean"]], axis=1, join='inner')

# Create PatchCollection and configure appearance
pc = PatchCollection(ipatches['Patches'], array=ipatches[f"{iname}_mean"], edgecolor='face', linewidth=1)
pc.set_clim([-0.1, 3])
ax.add_collection(pc)

# Polar plot formatting
ax.set_rlim([0, 90])
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
ax.set_title("Site 1a", y=1.08)

# Add colorbar
plt.colorbar(pc, ax=ax, location='bottom', shrink=0.6, pad=0.1, label='GNSS-VOD')

plt.tight_layout()
plt.show()


In [ ]:
vod.reset_index(inplace=True)
vod.head()

In [ ]:
#call patches
patches = pd.DataFrame({"CellID": patches.index, "Patches": patches.values})

# checking patches is a DataFrame
print("------ patches BEFORE animation ------")
print(patches)
print(type(patches))

assert isinstance(patches, pd.DataFrame), "patches must be a DataFrame"
assert "CellID" in patches.columns, "patches must have a 'CellID' column"

# normalizing epoch into date
vod['Epoch'] = pd.to_datetime(vod['Epoch'])
vod['Date'] = vod['Epoch'].dt.normalize()

unique_dates = sorted(vod['Date'].unique())

# figures
fig, ax = plt.subplots(1, 1, figsize=(8, 8), subplot_kw=dict(projection='polar'))

#empty pc
pc = None

#keeping the axes static
ax.set_rlim([0,90])
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)

def update(frame_date):
    global pc  # so we can reassign it
    date_str = pd.to_datetime(frame_date).strftime("%Y-%m-%d")
    fig.suptitle(f"Daily Mean GNSS-VOD 5.7 on {date_str}", fontsize=16)
    
    # Compute daily mean per CellID
    daily_avg = (
        vod.loc[vod['Date'] == frame_date]
        .groupby('CellID')['VOD1']
        .mean()
    )
    
    # check: show what this is
    print("\n------ daily_avg ------")
    print(daily_avg)
    print(type(daily_avg))

    # daily average df
    daily_avg = daily_avg.to_frame(name='VOD1_mean')

    # Join with patches
    ipatches = pd.concat(
        [patches.set_index('CellID'), daily_avg],
        join='inner',
        axis=1
    )

    # 
    if pc is not None:
        pc.remove()
    
    # created new patch collection
    pc = PatchCollection(
        ipatches['Patches'],
        array=ipatches['VOD1_mean'],
        edgecolor='face',
        linewidth=1,
        cmap='viridis'
    )
    pc.set_clim([-0.1, 3])

    ax.add_collection(pc)

    return [pc]

# Create animation
anim = FuncAnimation(
    fig,
    update,
    frames=unique_dates,
    blit=False
)

# Save animation
anim.save('/Volumes/coraldelmar/gnss_vod1_timeseries.gif', dpi=100, writer='pillow')
